In [ ]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import transformers 

Configuration

In [ ]:
max_length = 128 # Maximum Length of input sentences to the model 
batch_size =32 # Kích thước hàng loạt
epochs  = 2 # số kỷ nguyên huấn luyện mô hình 

# Labels in our dataset | các bộ loại nhãn cho dưc liệu mô hình [nhãn mâu thuẫn câu [0] , nhãn kế thừa câu[1] , nhãn trung lập [2]]
labels = ["contradiction" , "entailment", "neutral"]

Load the Data

In [ ]:
!curl -LO https://raw.githubusercontent.com/MohamadMerchant/SNLI/master/data.tar.gz
!tar -xvzf data.tar.gz

In [ ]:
# There are more than 500k samples in total: we will use 100k for this example. 
train_df = pd.read_csv("SNLI_Corpus/snli_1.0_train.csv", nrows=100000)
# Lấy ra validation data 
valid_df = pd.read_csv("SNLI_Corpus/snli_1.0_dev.csv")
# Lấy ra dữ liệu cho thử nghiệm mô hình 
test_df = pd.read_csv("SNLI_Corpus/snli_1.0_test.csv")


# Shape of the data in ra số lượng dữ liệu theo chiều hàng với mỗi 1 bộ =  1000 mẫu
print(f"Total train samples: {train_df.shape[0]}") 
print(f"Total validation samples: {valid_df.shape[0]}")
print(f"Total test samples: {valid_df.shape[0]}")

In [ ]:
print(f"sentence1: {train_df.iloc[1 , 'sentence1']}") 
print(f"sentence2: {train_df.iloc[1 , 'sentence2']}")
print(f"Similarity: {train_df.iloc[1 , 'similaryty']}")

In [ ]:
# We have some NaN entries(Những mục) in our train data, we will simply drop them
print("Num of missing values")
print(train_df.isnull().sum()) # In ra số lượng dữ liệu nan trên 3 bộ dữ liệu
train_df.dropna(axis=0 , inplace=True) # loại bỏ những dòng có dữ liệu NaN và thay thế nó 

In [ ]:
print('Train Target DisTribution')
print(train_df.similarity.value_counts())

In [ ]:
print("Validation Target Distribution")
print(valid_df.similarity.value_counts())

In [ ]:
# Thực hiện làm sạch dữ liệu loại bỏ đi các  dấu [-] đồng thời trộn số mẫu đặt lại chỉ số idx sau khi 
# xáo trộn chật tự
train_df = (
    train_df[train_df.similarity != '-']
    .sample(frac=1.0 , random_state=42)
    .reset_index(drop=True)
)

valid_df = (
    valid_df[valid_df.similarity != "-"]
    .sample(frac=1.0, random_state=42)
    .reset_index(drop=True)
)

In [ ]:
# Thiết lập nhãn đào tạo 
# tạo một cột mới thêm nó vào data Frame cột này sẽ có giá trị = 0 , 1 , 2
train_df['labels'] = train_df['similarity'].apply(
    lambda x : 0 if x == "contradiction" else 1 if x == "entailment" else 2

)
# Từ cột giá trị labels ta tách nó thành 3 cột riêng biệt 
y_train = tf.keras.utils.to_categorical(train_df.labels , num_classes=3)

valid_df ['labels'] = valid_df['similarity'].apply(
    lambda x : 0 if x == "contradiction" else 1 if x == "entailment" else 2
)
y_val = tf.keras.utils.to_categorical(valid_df.labels , num_classes=3)

test_df['labels'] = test_df['similarity'].apply(
    lambda x : 0  if x == 'contradiction' else 1 if x == 'entailment' else 2
)
y_test = tf.keras.utils.to_categorical(test_df.labels , num_classes=3)


In [ ]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data. [Tạo ra các lô của dữ liệu]
    
    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size. 
        shuffle: boolean, whether to shuffle the data. 
        include_targets: boolean, whether to include the labels .

    Returns: 
        Tuples '([input_ids , attention_mask, 'token_type_ids], labels)'
        (or just '[input_ids , attention_mask, 'token_type_ids]'if 'include_targets =False)
    """
    def __init__(self, 
        sentence_pairs,
        labels, 
        batch_size=batch_size,
        shuffle=True, 
        include_targets=True
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels 
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets

        # Load the Bert Tokenizer to encode the text. 
        # We will use base-base-uncased pretrained Model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased" , do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # biểu thi số lượng lô trên mỗi kỷ nguyên 
        return len(self.sentence_pairs) // self.batch_size 
    
    def __getitem__(self, index):
        # Truy xuất chỉ mục 
        # Tính chỉ số của các cặp câu trong lô nhân idx vơí batch_size
        # và lấy 1 đoạn con từ danh sách các chỉ số self.indexes
        indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
        # trả về một danh sách các cặp câu từ các chỉ số đã tính toán tương ứng 
        sentence_pairs = self.sentence_pairs[indexes]
        
        # With Bert tokenizer's batch_encode_plus batch of both the sentences are 
        # encoded togerther and separated (tách ly)  by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True ,
            max_length=max_length , return_attention_mask=True, 
            return_token_type_ids =True, 
            pad_to_max_length=True,
            return_tensors='tf',
        )

        #Chuyển đổi các lô của các đặc trưng được mã hóa thành mảng numpy 
        input_ids = np.array(encoded['input_ids'], dtype='int32')
        attention_mask = np.array(encoded['attention_mask'], dtype='int32')
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Đặt thành true nếu dữ liệu trình đào tạo dữ liệu được sử dụng cho huấn luyện và thẩm định 
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype='int32')
            return [input_ids , attention_mask , token_type_ids] , labels 

        else:
            return [input_ids, attention_mask, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        # Xáo trộn các chỉ mục 
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

Build the model 

In [ ]:
# Tạo mô hình theo phạm vi chiến lược phân phối 
strategy = tf.distribute.MirroredStrategy()

# Với sự tồn tại của phạm vi chiến lược 
with strategy.scope():
    # Mã hóa token_ids từ Bert tokenizer 
    input_ids = tf.keras.layers.Input(
        shape=(max_length,) , dtype=tf.int32 , name='input_ids'
    )
    # Attention masks indicates to the model which tokens should be attentioned to.
    attention_mask = tf.keras.layers.Input(
        shape=(max_length,) , dtype=tf.int32 , name='attention_mask'
    )
    # Token_type_ids are binary mask indentifying(nhận dạng) difderent sequences in the moel. 
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,) , dtype=tf.int32 , name='token_type_ids'
    )

    # Loading pretrained Bert model. 
    bert_model = transformers.TFBertModel.from_pretrained(
        "bert-base-uncased"
    )
    # Freeze the Bert model te reuse the pretrained features without modifying them. 
    bert_model.trainable = False 

    bert_output = bert_model.bert(
        input_ids , attention_mask=attention_mask , token_type_ids=token_type_ids
    )
    sequence_output = bert_output.last_hidden_state 
    pooled_output = bert_output.pooler_output 
    # Add trainable layers on top of frozen layers to adapt the pretrained features on the 
    # new data 
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64 , return_sequences=True)
    )(sequence_output)

    # Applying hyprid pooling approach to bi_lstm sequence output 
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)

    concat = tf.keras.layers.concatenate([avg_pool , max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    output = tf.keras.layers.Dense(3 , activation='softmax')(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids , attention_mask , token_type_ids] , outputs = output
    )
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr=0.001),
        loss = "categorical_crossentropy",
        metric = ['acc'],
    )

print(f"Strategy: {strategy}")
model.summary()


In [ ]:
train_data = BertSemanticDataGenerator(
    train_df[['sentence1' ,'sentence2']].values.astype('str'),
    y_train, 
    batch_size=batch_size, 
    shuffle =True,
)

valid_data = BertSemanticDataGenerator(
    valid_df[['sentence1' , 'sentence2']].values.astype("str"),
    y_val,
    batch_size=batch_size,
    shuffle=False,
)

Train the model Similarity estimate Bert


In [ ]:
# Xây dựng phương thức phù hợp cho Mô hình 
history = model.fit(
    # train , anh validation data
    train_data, 
    validation_data = valid_data,
    epochs = epochs , 
    # Ta sử dụng phương thức xử lý đa chiều 
    use_multiprocessing=True , 
    workers=-1,
)

Fine-Tuning and optimizer model.

In [ ]:
# Bỏ đóng băng bert để nó có thể được cập nhật số liệu 
bert_model.trainable = True 
# Biên dịch lại mô hình bằng việc làm thay đổi hiệu quả 
model.compile (
    optimizer=tf.keras.optimizers.Adam(1e-5),
    # sử dụng hàm loss crossentropy 
    loss = 'categorical_crossentropy',
    metrics=['accuracy'],
)
# xem tổng quát hóa mô hình 
model.summary()

Train the entire model end-to-end || Đào tạo toàn bộ mô hình từ đầu đến cuối 

In [ ]:
history = model.fit(
    train_data, 
    validation_data=valid_data, 
    epochs = epochs, 
    use_multiprocessing=True, 
    workers=-1,
)

Evaluate model on the test set

In [ ]:
test_data = BertSemanticDataGenerator(
    test_df[['sentence1', 'sentence2']].values.astype('str'),
    y_test , 
    batch_size=batch_size, 
    shuffle=False, 
)
model.evaluate(test_data, verbose=1)

Inference on custom sentences

In [ ]:
def check_similarity(sentence1 , sentence2):
    sentence_pairs = np.array([[str(sentence1) , str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs=sentence_pairs, labels=None , batch_size=1 ,
        shuffle=False , include_targets=False
    )
    proba = model.predict(test_data[0])[0]
    idx = np.argmax(proba)
    proba = f"{proba[idx]: .2f}%"
    pred = labels[idx]
    return pred, proba


In [ ]:
sentence1 = "Two women are observing something together."
sentence2 = "Two women are standing with their eyes closed."
check_similarity(sentence1, sentence2)

In [ ]:
sentence1 = "A soccer game with multiple males playing"
sentence2 = "Some men are playing a sport"
check_similarity(sentence1, sentence2)